# Azure AI Foundry o4-mini examples

<img src="https://www.cometapi.com/wp-content/uploads/2025/04/OpenAI-o4-mini-website-1024x576.webp" width=500>

We are excited to announce the general availability of the o3 and o4-mini models on Azure OpenAI Service. These models represent a significant step forward in AI capabilities, offering advanced reasoning and analysis for various applications.

- o3 Model: The o3 model is the most advanced reasoning model available on Azure OpenAI Service. It excels in complex problem-solving and multi-faceted analysis, making it ideal for tasks involving coding, math, science, and visual perception. It integrates various tools, including web browsing, Python code execution, image processing, and image generation 6. Additionally, it makes 20% fewer major errors compared to previous models on difficult, real-world tasks.

- o4-mini Model: The o4-mini model is a smaller, optimized version designed for fast, cost-efficient reasoning. It offers a competitive trade-off between price, speed, and performance, making it suitable for high-volume, high-throughput applications. It achieves high performance in math and coding task and performs well in visual tasks, including analyzing images and diagrams.

> https://azure.microsoft.com/en-us/blog/o3-and-o4-mini-unlock-enterprise-agent-workflows-with-next-level-reasoning-ai-with-azure-ai-foundry-and-github/?msockid=2e39c66c693c66a5151fd200687567d0

> https://openai.com/index/introducing-o3-and-o4-mini/

o4-mini:
- Context: 200K
- Max Output: 100K

In [ ]:
#%pip install azure-identity

In [9]:
import openai
import os
import sys
import tempfile
import time

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from datetime import datetime
from IPython.display import display, Markdown
from openai import AzureOpenAI
from pydantic import BaseModel, Field

In [10]:
print(f"Python version: {sys.version}")
print(f"OpenAI version: {openai.__version__}")

Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
OpenAI version: 1.88.0


In [11]:
print('OK') if load_dotenv("azure.env") else print('ERROR: Check file location or name.')

OK


In [12]:
print(f"Today is {datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 20-Jun-2025 16:49:09


## Settings

In [13]:
model = "o4-mini"  # As deployed in Azure AI Foundry

api_version = "2025-01-01-preview"  # Subject to change. Check: https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python-secure#api--feature-support

In [14]:
print(f"We will use {model} from Azure AI Foundry")

We will use o4-mini from Azure AI Foundry


In [15]:
def o4mini(prompt: str, reasoning_effort="medium"):
    """
    Sends a chat completion request to the Azure OpenAI API o4 mini model.
    Args:
        prompt (str): The input prompt to generate a response for.
    Returns:
        response (openai.Completion): The response object containing the generated completion.
    """
    try:
        # Azure OpenAI client
        start = time.time()

        aoai_client = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=api_version,
        )
        # Calling the model
        response = aoai_client.chat.completions.create(
            model=model,
            messages=[{
                "role": "user",
                "content": prompt
            }],
            max_completion_tokens=100_000,
            reasoning_effort=reasoning_effort,  # set to low, medium or high
        )

        elapsed = time.time() - start
        # Cost computing
        cost_input_token = 1.0470 / 1_000_000
        cost_output_token = 4.187884 / 1_000_000
        total_cost = (response.usage.prompt_tokens * cost_input_token) + (response.usage.completion_tokens * cost_output_token)
        # Elapsed time
        elapsed_time_msg = f"Completed in {time.strftime('%H:%M:%S' + str(elapsed % 1)[2:2], time.gmtime(elapsed))}"
        # Message generation
        msg_to_display = f"Prompt Tokens = {response.usage.prompt_tokens} | Completion tokens = {response.usage.completion_tokens} | Total tokens = {response.usage.total_tokens}\nTotal cost = {total_cost} EUR\n{elapsed_time_msg}\n"
        print(msg_to_display)

        return response

    except Exception as e:
        print(f"[ERROR] {e}")
        return None

## Quick test

In [16]:
prompt = "Who won the 2022 FIFA World Cup? Consider different perspectives, analyze the impact of the victory on global football, and discuss the team’s tactical approach in key matches."

resp = o4mini(prompt, reasoning_effort="low")

Prompt Tokens = 41 | Completion tokens = 1067 | Total tokens = 1108
Total cost = 0.004511399228 EUR
Completed in 00:00:13



In [17]:
print(resp.model_dump_json(indent=5))

{
     "id": "chatcmpl-BkU48qnpMuRZ2cA1JZnbgtPspEU37",
     "choices": [
          {
               "finish_reason": "stop",
               "index": 0,
               "logprobs": null,
               "message": {
                    "content": "The 2022 FIFA World Cup, held in Qatar from November 20 to December 18, was won by Argentina, who defeated France in a dramatic final that ended 3–3 after extra time and was decided 4–2 on penalties. Below is a multi-angle look at the victory, its wider impact, and Argentina’s tactical blueprint in their most important matches.\n\n1. Different Perspectives  \n  • Argentine fans and media  \n    – A first World Cup title in 36 years, widely hailed as the crowning glory of Lionel Messi’s career.  \n    – Nation-wide celebrations, massive public gatherings in Buenos Aires and across the country.  \n    – Political leaders spoke of unity and national pride, temporarily transcending partisan divides.  \n  • Global football community  \n    – Celebrat

In [18]:
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

The 2022 FIFA World Cup, held in Qatar from November 20 to December 18, was won by Argentina, who defeated France in a dramatic final that ended 3–3 after extra time and was decided 4–2 on penalties. Below is a multi-angle look at the victory, its wider impact, and Argentina’s tactical blueprint in their most important matches.

1. Different Perspectives  
  • Argentine fans and media  
    – A first World Cup title in 36 years, widely hailed as the crowning glory of Lionel Messi’s career.  
    – Nation-wide celebrations, massive public gatherings in Buenos Aires and across the country.  
    – Political leaders spoke of unity and national pride, temporarily transcending partisan divides.  
  • Global football community  
    – Celebrated as one of the all-time great finals—high drama, momentum swings, extra time and penalties.  
    – Cemented Messi’s status alongside Pelé and Maradona in the “greatest of all time” debate.  
    – Brought renewed attention to football’s capacity for 

In [19]:
resp.model

'o4-mini-2025-04-16'

## Basic examples

In [20]:
prompt = """
Imagine we are trying to figure out the employee hierarchy chart for the CONTOSO company with this:

Alan is the CEO of the company. Jane is the VP of Sales of EMEA. John is the global head of product development. Eric manages the shopping cart experience. Gilles manages sales in France. Sophie is responsible for sales for Italy. Marc reports to Sophie as well. Christine is in charge of user experience. Vincent is responsible of sales for Asia.
"""

resp = o4mini(prompt, reasoning_effort="low")
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 98 | Completion tokens = 908 | Total tokens = 1006
Total cost = 0.0039052046720000006 EUR
Completed in 00:00:12

Here’s one possible way to lay out the hierarchy based on the roles you’ve given:

Alan (CEO)  
├─ Jane (VP, Sales – EMEA)  
│   ├─ Gilles (Sales Manager, France)  
│   ├─ Sophie (Sales Manager, Italy)  
│   │   └─ Marc (Reports to Sophie)  
├─ Vincent (Sales Manager, Asia)  
└─ John (Global Head, Product Development)  
    ├─ Eric (Manager, Shopping Cart Experience)  
    └─ Christine (Lead, User Experience)


In [21]:
prompt = "What are the differences between o1, o3-mini and o4-mini genAI models in terms of size and complexity, performance, usecases and architecture?"

resp = o4mini(prompt, reasoning_effort="medium")
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 38 | Completion tokens = 1551 | Total tokens = 1589
Total cost = 0.006535194084000001 EUR
Completed in 00:00:14

Here’s a high-level comparison of the three “O-series” GenAI engines—O1, O3-mini and O4-mini—along four axes: size & complexity, raw performance, target use cases, and underlying architecture. (All figures below are illustrative/order-of-magnitude; your mileage may vary depending on tuning, quantization, etc.)

1. Size & Complexity  
   • O1  
     – Parameters: ~150–200 billion   
     – Layers: ~80–96 Transformer blocks   
     – Hidden dim: 12 k–16 k   
     – Model file (FP16): ~300 GB  
   • O3-mini  
     – Parameters: ~15–25 billion   
     – Layers: ~24–32 blocks   
     – Hidden dim: ~4 k–6 k   
     – Model file (FP16): ~30–40 GB  
   • O4-mini  
     – Parameters: ~30–40 billion   
     – Layers: ~32–48 blocks   
     – Hidden dim: ~6 k–8 k   
     – Model file (FP16): ~60–80 GB  

2. Performance (Throughput vs. Quality)  
   • O1  
     – Best-in-

### Legal example

In [22]:
prompt = """
You are an expert lawyer. So am I.
We are working on a document that will allow Sophie Martin to purchase a property for $710,000.
The property is located at 1500 Diagonal Road, Contoso City, California.
The sellers are John and Jane Smith. Sophie Martin will pay $5,000 in earnest money, and wants the right to conduct an inspection.
The parties should close within 60 days. This sale is not contingent on financing.

Could you help me draft the document?
"""

resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 110 | Completion tokens = 2733 | Total tokens = 2843
Total cost = 0.011560656972 EUR
Completed in 00:00:55

Below is a simple, California‐style real estate purchase agreement tailored to your facts.  You and your client should review and adapt it as needed, and insert any exhibits (e.g., legal description, disclosures) before execution.

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––  
REAL ESTATE PURCHASE AGREEMENT  
1500 Diagonal Road, Contoso City, California  
Effective Date: ____________, 20__  
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

1.  PARTIES  
    1.1  Seller:  John Smith and Jane Smith (“Seller”).  
    1.2  Buyer:   Sophie Martin (“Buyer”).  

2.  PROPERTY  
    2.1  Address: 1500 Diagonal Road, Contoso City, California.  
    2.2  Legal Description:  Attached as Exhibit A.  

3.  PURCHASE PRICE  
    3.1  Total Price: $710,000.00.  
    3.2  Payment:  
         a)  Earnest Money Deposit: $5,000

In [23]:
base_prompt = (
    "<prompt>You are a lawyer specializing in competition law, "
    "assisting business owners with their questions.</prompt>\n"
    "<policy>As a legal professional, provide clear and accurate "
    "information about competition law while maintaining "
    "confidentiality and professionalism. Avoid giving specific "
    "legal advice without sufficient context, and encourage clients "
    "to seek personalized counsel when necessary. Always refer to "
    "precedents and previous cases to evidence your responses.</policy>\n")

legal_query = (
    "<query>A larger company is offering suppliers incentives not to do "
    "business with me. Is this legal?</query>")

In [24]:
example_prompt = (
    "<prompt>You are a lawyer specializing in competition law, "
    "assisting business owners with their questions.</prompt>\n"
    "<policy>As a legal professional, provide clear and accurate "
    "information about competition law while maintaining "
    "confidentiality and professionalism. Avoid giving specific "
    "legal advice without sufficient context, and encourage clients "
    "to seek personalized counsel when necessary.</policy>\n"
    """<example>
<question>
I'm considering collaborating with a competitor on a joint marketing campaign. Are there any antitrust issues I should be aware of?
</question>
<response>
Collaborating with a competitor on a joint marketing campaign can raise antitrust concerns under U.S. antitrust laws, particularly the Sherman Antitrust Act of 1890 (15 U.S.C. §§ 1–7). Section 1 of the Sherman Act prohibits any contract, combination, or conspiracy that unreasonably restrains trade or commerce among the states.

**Key Considerations:**

1. **Per Se Illegal Agreements:** Certain collaborations are considered automatically illegal ("per se" violations), such as price-fixing, bid-rigging, and market allocation agreements. For example, in *United States v. Topco Associates, Inc.*, 405 U.S. 596 (1972), the Supreme Court held that market division agreements between competitors are per se illegal under the Sherman Act.

2. **Rule of Reason Analysis:** Collaborations that are not per se illegal are evaluated under the "rule of reason," which assesses whether the pro-competitive benefits outweigh the anti-competitive effects. In *Broadcast Music, Inc. v. Columbia Broadcasting System, Inc.*, 441 U.S. 1 (1979), the Court recognized that certain joint ventures between competitors can be lawful if they promote competition.

3. **Information Sharing Risks:** Sharing competitively sensitive information, such as pricing strategies or customer data, can lead to antitrust violations. The Department of Justice and the Federal Trade Commission caution against exchanges that could facilitate collusion (*Antitrust Guidelines for Collaborations Among Competitors*, 2000).

**Recommendations:**

- **Define the Scope:** Clearly delineate the parameters of the collaboration to focus on the marketing campaign without involving competitive aspects like pricing or market division.
- **Implement Safeguards:** Establish protocols to prevent the exchange of sensitive information that is not essential to the marketing effort.
- **Legal Consultation:** Given the complexities of antitrust laws, consult with a legal professional to ensure the collaboration complies with all legal requirements.

**Conclusion:**

While joint marketing campaigns between competitors are not inherently illegal, they must be structured carefully to avoid antitrust pitfalls. Legal guidance is essential to navigate these issues and to design a collaboration that achieves your business objectives without violating antitrust laws.
</response>
</example>""")

In [25]:
prompt = example_prompt + legal_query
print(prompt)

<prompt>You are a lawyer specializing in competition law, assisting business owners with their questions.</prompt>
<policy>As a legal professional, provide clear and accurate information about competition law while maintaining confidentiality and professionalism. Avoid giving specific legal advice without sufficient context, and encourage clients to seek personalized counsel when necessary.</policy>
<example>
<question>
I'm considering collaborating with a competitor on a joint marketing campaign. Are there any antitrust issues I should be aware of?
</question>
<response>
Collaborating with a competitor on a joint marketing campaign can raise antitrust concerns under U.S. antitrust laws, particularly the Sherman Antitrust Act of 1890 (15 U.S.C. §§ 1–7). Section 1 of the Sherman Act prohibits any contract, combination, or conspiracy that unreasonably restrains trade or commerce among the states.

**Key Considerations:**

1. **Per Se Illegal Agreements:** Certain collaborations are consi

In [26]:
resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 572 | Completion tokens = 1591 | Total tokens = 2163
Total cost = 0.007261807444000001 EUR
Completed in 00:00:18

Under U.S. and EU competition laws, a dominant or strong buyer’s use of exclusive‐dealing incentives to keep suppliers from dealing with a rival can—under the right circumstances—be unlawful.  Whether it crosses the line depends largely on (1) the degree of the buyer’s market power, (2) how much of the available supply is foreclosed, and (3) the nature and size of the incentive.

1. U.S. Law  
   • Sherman Act § 1 and Clayton Act § 3:  Exclusive‐dealing arrangements aren’t “per se” illegal, but they are judged under the “rule of reason.”  Courts look at whether the arrangement unreasonably restrains trade by foreclosing a substantial share of the market.  
   • Key factors:  
     – Buyer’s market share or ability to control price/distribution in the relevant market.  
     – Percentage of total supplier capacity or sales foreclosed by the incentive.  Courts

## Drawing

In [27]:
prompt = """
Generate an image of a cat.
"""

resp = o4mini(prompt)
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 14 | Completion tokens = 656 | Total tokens = 670
Total cost = 0.002761909904 EUR
Completed in 00:00:10

Here’s a simple ASCII “image” of a cat:

        /\_/\  
       ( o.o ) 
        > ^ <  

And here’s a more detailed one:

           /\     /\
          {  `---'  }
          {  O   O  }
          ~~>  V  <~~
           \  \|/  /
            `-----'____
            /     \    \_
           {       }\  )_\_   _
           |  \_/  |/ /  \_\_/ )
            \__/  /(_/     \__/
              (__/


## Maths


In [28]:
prompt = """
Determine h(t) given that h"(t) = 24t2 - 48t + 2, h(1) = - 9 and h(-2) = - 4
"""

resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 48 | Completion tokens = 2140 | Total tokens = 2188
Total cost = 0.009012327760000001 EUR
Completed in 00:00:18

First integrate twice:

1) h′(t)=∫(24 t²−48 t+2) dt=8 t³−24 t²+2 t+C₁  
2) h(t)=∫h′(t) dt=2 t⁴−8 t³+ t² +C₁ t+C₂  

Use the two conditions:

h(1)=2−8+1 +C₁+ C₂=−5 +C₁+ C₂=−9 ⇒ C₁+ C₂=−4  
h(−2)=2·16−8·(−8)+4 −2 C₁+ C₂=32+64+4 −2 C₁+ C₂=100−2 C₁+ C₂=−4 ⇒ −2 C₁+ C₂=−104  

Solve:
 −2 C₁+ C₂=−104  
  C₁+ C₂=−4  
⟹ subtract: (−2 C₁+ C₂)−(C₁+ C₂)=−104−(−4)=−100  
      −3 C₁=−100 ⇒ C₁=100/3  
      C₂=−4−C₁=−4−100/3=−112/3  

Hence

h(t)=2 t⁴−8 t³+ t² + (100/3) t −112/3.


In [29]:
prompt = "For which natural numbers 𝑛 is the number 𝑁2=𝑛4-3𝑛2+9 a prime number?"

resp = o4mini(prompt, reasoning_effort='medium')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 39 | Completion tokens = 2393 | Total tokens = 2432
Total cost = 0.010062439412000001 EUR
Completed in 00:00:29

Write  
 N₂(n) = n⁴ − 3n² + 9.  
Observe the factorization over ℤ:  
 n⁴ − 3n² + 9 = (n² − 3n + 3)(n² + 3n + 3).  
For n a positive integer both factors exceed 1 as soon as n ≥ 3.  
The only solutions of n² − 3n + 3 = 1 are n = 1, 2.  Hence:

• n = 1: N₂ = 1⁴ − 3·1² + 9 = 7 (prime)  
• n = 2: N₂ = 2⁴ − 3·2² + 9 = 13 (prime)

For every n ≥ 3 both factors are >1, so N₂ is composite.  
Answer: n = 1 or 2.


## Coding

In [30]:
prompt = "Can you provide a structured template and guide for setting up a Python Flask project that connects to a PostgreSQL database?"

resp = o4mini(prompt, reasoning_effort='medium')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 29 | Completion tokens = 1840 | Total tokens = 1869
Total cost = 0.007736069560000001 EUR
Completed in 00:00:24

Below is a step-by-step guide plus a “skeleton” template you can copy/adjust to kick off a Flask + PostgreSQL project using Flask-SQLAlchemy and Flask-Migrate.

1. Prerequisites  
   • Python 3.7+  
   • PostgreSQL server running (and a database created)  
   • (Optionally) Docker  

2. Create & activate a virtualenv  
   ```bash
   python3 -m venv venv
   source venv/bin/activate        # macOS/Linux
   venv\Scripts\activate           # Windows
   ```

3. Install dependencies  
   ```bash
   pip install flask
   pip install flask_sqlalchemy
   pip install psycopg2-binary     # Postgres driver
   pip install flask_migrate
   pip install python-dotenv       # load .env files
   ```

4. Layout your project  
   ```
   myflaskapp/
   ├── app/
   │   ├── __init__.py
   │   ├── config.py
   │   ├── extensions.py
   │   ├── models.py
   │   └── routes.py
   ├── mig

## Solving

In [31]:
prompt = "Solve for x in the equation 3x^2 + 5x - 2 = 0"

resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 27 | Completion tokens = 375 | Total tokens = 402
Total cost = 0.0015987255000000002 EUR
Completed in 00:00:06

You can factor or use the quadratic formula.  Factoring:  
3x² + 5x – 2 = (3x – 1)(x + 2) = 0  
⟹ 3x – 1 = 0  ⇒  x = 1/3  
⟹ x + 2 = 0   ⇒  x = –2  

So the solutions are  
x = 1/3  or  x = –2.


## Physic example

In [32]:
prompt = """ 
A ball is thrown vertically upward at the same instant that a second ball is dropped from rest directly above it.
The two balls are 12.0 meters apart when they start their motion.

Find the maximum speed at which the first ball can be throw such that it doesn't collide with the second ball before it returns to its starting height. Treat the balls as being very small (i.e. ignore their diameters).
"""

resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 90 | Completion tokens = 4466 | Total tokens = 4556
Total cost = 0.018797319944000003 EUR
Completed in 00:00:56

Let y=0 be the launch‐height of ball 1 and up be +.  Then

  ball 1:  y₁(t)=v₀ t−½ g t²  
  ball 2:  y₂(t)=H −½ g t²  

A collision requires y₁=y₂ ⇒ v₀ t=H ⇒ t_c=H/v₀.  Ball 1 returns to y=0 at  
   t_return=2 v₀/g.  

To avoid any collision before return we demand  
   t_c ≥ t_return  
⇔ H/v₀ ≥ 2 v₀/g  
⇔ v₀² ≤ (g H)/2  
⇔ v₀ ≤ √(g H/2).  

With H=12.0 m and g=9.80 m/s²,  
   v₀,max=√[9.80·12.0/2]≈√58.8≈7.7 m/s.


## Optimization

In [33]:
prompt = """
x and y are integers.
You need to maximize this function: 3x + 2y

with these five contraints:
2x + y <= 18
2x + 3y <= 42
3x + y <= 24
x >= 0
y >= 0

Print the results of this optimal problem.
"""

resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 78 | Completion tokens = 2054 | Total tokens = 2132
Total cost = 0.008683579736 EUR
Completed in 00:00:43

The maximum is attained at  
  x = 3, y = 12  
giving  
  3·x + 2·y = 3·3 + 2·12 = 33.


In [34]:
prompt = """
300 kids need to travel to the Bronx zoo.
The school may rent 40 seats and 30 seats buses for $500 and $400.

How many buses of each size do you use to minimize cost?
Print the number of 40 seats buses and the number of 30 seats buses and the total cost.
"""

resp = o4mini(prompt)
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 70 | Completion tokens = 856 | Total tokens = 926
Total cost = 0.003658118704000001 EUR
Completed in 00:00:19

Let x = number of 40-seat buses, y = number of 30-seat buses.  
We need  
  40x + 30y ≥ 300  
Minimize cost C = 500x + 400y  

Checking integer solutions:  
x=6 ⇒ capacity 6·40=240, need 60 more ⇒ y=2 ⇒ C=6·500+2·400=3000+800=3800  
That meets exactly 300 seats and is the cheapest possible.  

Answer:  
40-seat buses: 6  
30-seat buses: 2  
Total cost: $3,800


In [35]:
prompt = """
A train X left at 11:58 and travels at a speed of 210 km/h.
A train Y left at 13:10 but travels at 380 km/h.

At what time does train Y catch up with train X?
"""

resp = o4mini(prompt)
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 56 | Completion tokens = 997 | Total tokens = 1053
Total cost = 0.004233952348000001 EUR
Completed in 00:00:23

Let t be the time in hours after 11 :58 that X has been running when Y catches up.  Since Y starts 1 h 12 min = 1.2 h later, its running time is t–1.2 (for t≥1.2).  We set distances equal:

210·t = 380·(t–1.2)

⇒ 210t = 380t – 456  
⇒ 380t – 210t = 456  
⇒ 170t = 456  
⇒ t = 456/170 ≈ 2.68235 h

2.68235 h = 2 h + 0.68235·60 min ≈ 2 h 40 min 56 s  

So the catch-up time is  
11 :58 + 2 h 40 min 56 s ≃ 14 :38 56 ≃ 2 :39 PM.


## Chemical

In [36]:
prompt = """
Sodium hydroxide (NaOH) reacts with phosphoric acid (H₃PO₄) to form sodium phosphate (Na₃PO₄) and water (H₂O) by the reaction:
3 NaOH(aq) + H₃PO₄(aq) → Na₃PO₄(aq) + 3 H₂O(l) If 35.60 grams of NaOH are reacted with 30.80 grams of H₃PO₄.
How many grams of Na₃PO₄ are formed?
"""

resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 123 | Completion tokens = 1035 | Total tokens = 1158
Total cost = 0.00446324094 EUR
Completed in 00:00:21

First, calculate moles of each reactant:

• Molar mass NaOH ≃ 40.00 g/mol  
  n(NaOH) = 35.60 g ÷ 40.00 g/mol = 0.890 mol  

• Molar mass H₃PO₄ ≃ 98.00 g/mol  
  n(H₃PO₄) = 30.80 g ÷ 98.00 g/mol = 0.314 mol  

The reaction requires 3 mol NaOH per 1 mol H₃PO₄.  
Check which is limiting:  
• To consume 0.314 mol H₃PO₄ you’d need 3×0.314 = 0.942 mol NaOH, but you have only 0.890 mol → NaOH is limiting.  

Moles Na₃PO₄ produced = n(NaOH)/3 = 0.890 mol ÷ 3 = 0.297 mol  

Molar mass Na₃PO₄ = 3·22.99 + 30.97 + 4·16.00 = 163.94 g/mol  

Mass Na₃PO₄ = 0.297 mol × 163.94 g/mol ≃ 48.6 g  

Answer: ≃ 48.6 g of Na₃PO₄ can be formed.


## Predictive maintenance

In [37]:
prompt = """
Proactive Maintenance Planning for Contoso: Advanced Transformer Analysis

Background
Contoso, a leading energy provider, is undertaking a comprehensive analysis of its electrical distribution network to optimize maintenance planning for its critical transformers. The goal is to proactively identify transformers at risk of failure due to thermal stress, insulation degradation, overloading, and other factors, especially under projected future operating conditions. This analysis requires detailed examination of operational parameters, environmental conditions, diagnostic test results, and adherence to safety regulations.
 
Datasets
Overview
 
You are provided with detailed datasets for seven transformers (T1001 to T1007). Your task is to analyze these datasets to determine which transformer(s) require immediate maintenance or action within the next six months. The data includes:

Asset Inventory and Specifications
Load and Operational Data
Environmental Conditions
Transformer Thermal Performance Data
Insulation Deterioration Rates
Transformer Loading Guide
Technician Reports
Dissolved Gas Analysis (DGA) Results
Safety Regulations and Maintenance Constraints
Projected Future Operations
 
Dataset 1: Asset Inventory and Specifications
 
Asset Inventory Table
Asset ID	Location	Install Date	Expected Lifespan (years)	Last Maintenance Date	Manufacturer	Model	Voltage Ratio (kV/kV)	Power Rating (MVA)	Impedance (%)	Cooling Type	Criticality Score (1-10)	Expected Retirement Date
T1001	123 Main St	2005-06-15	25	2018-05-20	ElectroTrans Co.	ETX-2000	220/110	100	8	ONAN	9	2030-06-15
T1002	456 Oak Ave	2010-08-22	25	2017-11-10	PowerMax Solutions	PMX-1500	220/110	90	9	ONAF	7	2035-08-22
T1003	789 Industrial Rd	2000-03-30	25	2015-09-05	VoltGear Ltd.	VG-3000	220/110	120	10	ONAN	10	2025-03-30
T1004	321 Maple Dr	1995-01-12	25	2014-04-18	ElectroTrans Co.	ETX-1800	110/33	80	7	ONAN	6	2020-01-12
T1005	654 Pine St	2007-12-05	25	2016-07-25	PowerMax Solutions	PMX-1600	220/110	109	6	ONAF	8	2032-12-05
T1006	987 Elm St	1990-11-20	25	2013-03-15	VoltGear Ltd.	VG-2500	220/110	90	8	ONAN	5	2015-11-20
T1007	222 Cedar Rd	2008-09-10	25	2019-02-22	ElectroTrans Co.	ETX-2200	220/110	110	7	ONAF	8	2033-09-10
 												
Additional Notes:												
 												
Impedance (%) affects voltage regulation and fault levels.
Cooling Types:
ONAN: Oil Natural Air Natural
ONAF: Oil Natural Air Forced
 
Dataset 2: Load and Operational Data
Monthly Average Load Data (Last 12 Months)
 
Transformer T1001
 
Month	Average Load (MVA)
Month -11	80
Month -10	82
Month -9	84
Month -8	85
Month -7	87
Month -6	88
Month -5	90
Month -4	92
Month -3	90
Month -2	88
Month -1	90
Current Month	92
 	
---	
 	
#### Transformer T1002	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	66
Month -10	67
Month -9	68
Month -8	69
Month -7	68
Month -6	70
Month -5	71
Month -4	70
Month -3	69
Month -2	70
Month -1	69
Current Month	70
 	
---	
 	
#### Transformer T1003	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	110
Month -10	112
Month -9	114
Month -8	116
Month -7	116
Month -6	116
Month -5	116
Month -4	116
Month -3	118
Month -2	115
Month -1	116
Current Month	117
 	
---	
 	
#### Transformer T1004	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	60
Month -10	61
Month -9	62
Month -8	63
Month -7	62
Month -6	64
Month -5	65
Month -4	66
Month -3	65
Month -2	64
Month -1	65
Current Month	65
 	
---	
 	
#### Transformer T1005	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	90
Month -10	91
Month -9	92
Month -8	93
Month -7	94
Month -6	95
Month -5	96
Month -4	95
Month -3	96
Month -2	95
Month -1	95
Current Month	95
 	
---	
 	
#### Transformer T1006	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	88
Month -10	88
Month -9	88
Month -8	87
Month -7	87
Month -6	86
Month -5	86
Month -4	85
Month -3	85
Month -2	85
Month -1	85
Current Month	85
 	
---	
 	
#### Transformer T1007	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	95
Month -10	96
Month -9	97
Month -8	98
Month -7	99
Month -6	100
Month -5	101
Month -4	100
Month -3	99
Month -2	100
Month -1	100
Current Month	100
 	
 
Projected Load Changes:
T1001: Expected to remain relatively stable with minor fluctuations due to seasonal demand variations.
T1002: Anticipated to maintain current load levels with minor fluctuations. However, a significant load increase of 15 MVA is expected in Month +4 due to the connection of a new residential development.
T1003: Expected to increase by 1% per month due to new industrial expansion starting immediately.
T1004: Expected to remain relatively stable with minor fluctuations, maintaining its current load levels.
T1006: Expected to experience a slight decrease in load by -1% per month due to energy efficiency programs implemented in the service area.
T1007: Anticipated to maintain relatively stable load levels with minor fluctuations typical of regular operational demands.
 
Notes:

The transformers T1001, T1002 (until Month +4), T1004, T1005, and T1007 are expected to have stable load profiles, with only minor fluctuations that are characteristic of normal seasonal variations and operational demands.
T1002 will experience a significant load increase in Month +4 due to the upcoming new residential development, which is projected to add 15 MVA to its current load.
The load increase for T1003 due to industrial expansion and the decrease for T1006 due to energy efficiency measures are significant and should be considered in operational planning.
All projections consider current known factors; unexpected changes in consumer behavior or unforeseen events are not accounted for in these projections.


Dataset 3: Environmental Conditions
Ambient Temperature Data (Historical and Forecasted)

Ambient Temperatures (ÃÂ¡C):

Month	Ambient Temp (ÃÂ¡C)
Month -11	25
...	...
Current Month	40
Month +1	41
Month +2	42
Month +3	43
Month +4	42
Month +5	40
Month +6	38
 	
Notes:	
 	
Ambient temperatures peak in Month +3.
High temperatures can influence transformer cooling efficiency and insulation aging.
 
Dataset 4: Transformer Thermal Performance Data
Cooling Efficiency Coefficients

Model	Cooling Type	CEC	Rated ___TO-R (ÃÂ¡C)	Rated ___HS-R (ÃÂ¡C)
VG-3000	ONAN	1.0	55	30
PMX-1500	ONAF	0.85	50	25
ETX-2000	ONAN	0.95	55	30
VG-2500	ONAN	0.9	55	30
PMX-1600	ONAF	0.88	50	25
ETX-1800	ONAN	0.92	55	30
ETX-2200	ONAF	0.86	50	25
 				
Exponents for Temperature Calculations				
 				
ONAN Cooling:
( m = 0.8 )
( n = 1.6 )
ONAF Cooling:
( m = 0.9 )
( n = 1.8 )
 
Dataset 5: Insulation Deterioration Rates
Insulation Life Expectancy vs. Hot-Spot Temperature

Hot-Spot Temperature (ÃÂ¡C)	Relative Aging Rate
110	1
120	2
130	4
140	8
150	16
 	
---	
 	
### Dataset 6: Transformer Loading Guide	
	
IEEE Standard Formulas	
 	
Top-Oil Temperature Rise (( \Delta \theta_{\text{TO}} )):

[
\Delta \theta_{\text{TO}} = \Delta \theta_{\text{TO-R}} \times \left( \frac{L}{L_{\text{R}}} \right)^{m}
]

Hot-Spot Temperature Rise (( \Delta \theta_{\text{HS}} )):

[
\Delta \theta_{\text{HS}} = \Delta \theta_{\text{HS-R}} \times \left( \frac{L}{L_{\text{R}}} \right)^{n}
]

Total Hot-Spot Temperature (( \theta_{\text{HS}} )):

[
\theta_{\text{HS}} = \theta_{\text{Amb}} + \Delta \theta_{\text{TO}} + \Delta \theta_{\text{HS}}
]

Where:

( L ): Load in MVA
( L_{\text{R}} ): Rated Load in MVA
( m ), ( n ): Exponents based on cooling type
 

Dataset 7: Technician Reports

T1001 Report:

No significant issues observed. Regular operational parameters.

T1002 Report:
  Cooling fans operating efficiently. Oil color slightly darker than normal; recommend oil test.  

T1003 Report:
  Transformer oil shows higher than normal moisture content.  
  DGA indicates increased levels of ethylene and methane.  
  Cooling system operating at maximum capacity.  

T1004 Report:
  Signs of aging observed. Minor oil leaks detected.  

T1005 Report:
  External overheating signs on components.  
  Load readings higher during peak hours.  

T1006 Report:
  Transformer operating past expected lifespan.  
  Cooling fins corroded; reduced cooling efficiency expected.  

T1007 Report:
  All parameters within normal limits. No action required.  
 

Dataset 8: Dissolved Gas Analysis (DGA) Results
 
Gas Levels in Parts Per Million (ppm)

Gas	T1001	T1002	T1003	T1004	T1005	T1006	T1007
Hydrogen (H2)	20	40	150	30	50	80	25
Methane (CH4)	10	25	120	15	30	60	12
Ethylene (C2H4)	5	15	90	8	20	40	5
Ethane (C2H6)	4	10	60	6	15	35	4
Acetylene (C2H2)	0	0	5	0	0	0	0
Carbon Monoxide (CO)	50	80	300	70	90	200	60
Carbon Dioxide (CO2)	200	400	1200	250	450	800	220
 							
Interpretation Standards:							
 							
Action Levels:
H2 > 100 ppm
CH4 > 75 ppm
C2H4 > 50 ppm
CO > 350 ppm
Presence of Acetylene (C2H2) indicates high-temperature arcing.
High levels of CO and CO2 suggest cellulose insulation degradation.
 
Dataset 9: Safety Regulations and Maintenance Constraints
 
Regulatory Guidelines:

Maintenance must not be performed when ambient temperatures exceed 40ÃÂ¡C due to safety concerns.
Transformers operating beyond 80% of their expected lifespan require thorough inspections.
DGA action levels necessitate immediate investigation and potential maintenance.
Ambient Temperature Maintenance Window: Best scheduled during months with ambient temperatures below 40ÃÂ¡C.

Dataset 11: SME reports
Subject: Analysis of Time-Variant Load Impact on Transformer T1005 Due to New Industrial Park Opening
 
Establishments Overview:
 

MetalWorks Manufacturing Co.
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
GreenTech Recycling Facility
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
FreshFoods Processing Plant
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
LogiTrans Distribution Center
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Quantum Data Centers Ltd.
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging
 
Time-Variant Load Profiles:
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):
Time Window	MetalWorks (MW)	GreenTech (GT)	FreshFoods (FF)	LogiTrans (LT)	Quantum Data (QD)
1 (00-04)	1.0	0.5	0.0	0.0	2.5
2 (04-08)	2.0	1.0	0.0	0.0	2.5
3 (08-12)	4.0	3.0	2.5	0.0	3.0
4 (12-16)	4.0	3.0	2.5	3.5	3.0
5 (16-20)	2.0	1.5	1.0	3.5	2.0
6 (20-24)	1.0	0.5	0.0	1.0	2.5
 					

1. Upcoming Commercial Complex Affecting Transformer T1001
 
Establishments Overview:

Sunrise Retail Plaza
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
EcoWise Offices
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
Harvest Foods Market
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
Swift Logistics Hub
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Nimbus Data Centers
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging

Time-Variant Load Profiles:
 
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):
Time Window	Sunrise Retail	EcoWise Offices	Harvest Foods	Swift Logistics	Nimbus Data
1 (00-04)	0.5	0.3	0.0	0.0	0.8
2 (04-08)	0.8	0.5	0.0	0.0	0.8
3 (08-12)	1.0	0.6	0.5	0.0	0.9
4 (12-16)	1.0	0.6	0.5	0.5	0.9
5 (16-20)	0.6	0.5	0.2	0.5	0.6
6 (20-24)	0.5	0.3	0.0	0.2	0.8
 					
---

2. Residential Development Impacting Transformer T1002
 
Establishments Overview:
 
Grandview Shopping Center
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
Harmony Business Park
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
FreshMart Grocery
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
Express Logistics Depot
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Apex Data Center
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging

Time-Variant Load Profiles:
 
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):
Time Window	Grandview Mall	Harmony Business	FreshMart Grocery	Express Logistics	Apex Data Center
1 (00-04)	0.5	0.3	0.0	0.0	0.8
2 (04-08)	0.8	0.5	0.0	0.0	0.8
3 (08-12)	1.0	0.6	0.5	0.0	0.9
4 (12-16)	1.0	0.6	0.5	0.5	0.9
5 (16-20)	0.6	0.5	0.2	0.5	0.6
6 (20-24)	0.5	0.3	0.0	0.2	0.8
 					
---		

3. Small Industrial Unit Connecting to Transformer T1004
 
Overview:

A small-scale manufacturing unit is set to commence operations near Transformer T1004:

Precision Tools Ltd.: Specializing in custom tool manufacturing.
Expected Load Increase: Approximately 1.8 MVA.

4. New School Facility Supplied by Transformer T1005
 
Overview:
A new educational institution is opening in the area served by Transformer T1005:
Sunrise Elementary School: Modern facilities with standard energy consumption.
Expected Load Increase: Approximately 1.2 MVA.

5. Community Center Addition Affecting Transformer T1007
 
Overview:
Construction of a community center is nearing completion in the vicinity of Transformer T1007:
Riverdale Community Hall: Hosting events and activities for local residents.
Expected Load Increase: Approximately 1 MVA.

6. Small Business Cluster Connecting to Transformer T1003
 
Overview:
A cluster of small businesses is opening near Transformer T1003:

Shops Include:
Cafe Delight: Boutique coffee shop.
Health Rx Pharmacy: Local pharmacy outlet.
Bloom Floral Shop: Florist with refrigeration needs.
TechFix IT Services: Electronics repair shop.
FitLife Gym Studio: Fitness center with standard equipment.
Expected Load Increase: Approximately 2 MVA combined.
 

Question
Task
 
As an analyst at Contoso, your tasks are:

Identify the transformer(s) that should be prioritized for maintenance or immediate action within the next six months, based on the data provided.
For each transformer you identify, provide a detailed rationale explaining why it should be prioritized, including all necessary calculations, engineering considerations, and interpretations of the DGA results.
Recommend a specific maintenance or replacement plan for these transformer(s), considering operational constraints, safety regulations, projected load changes, and potential risks.

Note: Your analysis should aim to:
Prevent potential transformer failures due to thermal faults, insulation degradation, or overloading.
Utilize electrical engineering principles and standards for transformer diagnostics.
Ensure compliance with safety regulations and operational constraints.
Optimize maintenance scheduling by considering ambient temperature constraints and load projections.
Focus on one or two transformers that critically need attention based on the data.
 
Instructions:
Thoroughly analyze the provided datasets, performing all necessary calculations to assess the condition and future performance of the transformers.
Apply electrical engineering principles, including calculations of hot-spot temperatures, interpretation of DGA results, and understanding of thermal impacts on transformer insulation.
Include all intermediate calculations and reasoning steps in your analysis.
Reference specific data points from the datasets in your rationale.
Provide a brief executive summary at the beginning of your solution, highlighting your key findings and recommendations.
Present your findings in a clear, logical, and concise manner, suitable for presentation to the management team.
Identify only the transformer(s) that genuinely require immediate action, ensuring that your conclusions are based on solid evidence from the data.
  
"""

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\vipottur\AppData\Local\Temp\ipykernel_20308\3446726394.py:1: SyntaxWarning: invalid escape sequence '\D'
  prompt = """


In [38]:
resp = o4mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 4899 | Completion tokens = 5208 | Total tokens = 10107
Total cost = 0.026939752872000004 EUR
Completed in 00:01:24

Executive Summary  
After a detailed review of operational data, thermal‐life calculations, DGA results and lifecycle status, two transformers emerge as the highest priority for action within the next six months:  
1. T1003 (VG-3000) – Immediate critical action required  
2. T1006 (VG-2500) – Near-term replacement/overhaul  

———  
1. Transformer T1003 (VG-3000, ONAN, Rated 120 MVA)  
Condition Highlights  
 • Age & Criticality: Installed 2000 (23 years old), expected end-of-life 2025, criticality = 10  
 • Loading: Current 117 MVA (97.5% of rating). Projected +1%/month → in 6 months ~124 MVA (104% of rating)  
 • Ambient: 40 °C now, peaking at 43 °C in Month +3  
 • DGA: H₂=150 ppm (>100 action), CH₄=120 ppm (>75), C₂H₄=90 ppm (>50), CO₂=1200 ppm → cellulose degradation  
 • Moisture: Technician report “higher than normal moisture”  
 • Cooling: Operating